# Python notebooks on Microsoft Fabric

## Topic of notebook

### Install and import libraries

**!pip install** = Shell command that installs the library on the _Driver Node_ only.

**%pip install** = Magic command that installs the library on _Driver and Worker Nodes_ (Spark)

In [ ]:
!pip install semantic-link-labs -qq


In [ ]:
import notebookutils as nbutils
import logging
import sys

import polars as pl
import pyarrow as pa
import sempy.fabric as fabric
import sempy_labs as slab

### Set up logging

In [ ]:
logging.basicConfig(level=logging.WARNING)

# Use a named logger
nb_logger = logging.getLogger("notebook_logger")
nb_logger.setLevel(logging.INFO)

# Add handler and formattter
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s %(levelname)-5s %(message)s", datefmt="%H:%M:%S")

handler.setFormatter(formatter)
nb_logger.addHandler(handler)
nb_logger.propagate = False

# Silence all other loggers
for name, logger in logging.root.manager.loggerDict.items():
    if name != "notebook_logger":
        logging.getLogger(name).setLevel(logging.WARNING)


**General imports**

In [ ]:
import logging
import time
from typing import Any

import pandas as pd
import pyarrow as pa

logger = logging.getLogger(__name__)


# TODO: Observe for coming changes to Polars implementation
def excel_to_arrow_table(excel_abfss_path: str, sheet_name: str, preserve_index: bool = False, **kwargs: Any) -> pa.Table:
    """Returns a pyarrow table after reading a specific sheet in the Excel file.

    Additional keyword arguments are passed directly to 'pd.read_excel'.
    """
    try:
        # Pass sheet_name explicitly to keep signature clear,
        # merge with any additional kwargs (kwargs can override sheet_name if given).
        pandas_df = pd.read_excel(excel_abfss_path, sheet_name=sheet_name, engine="openpyxl", **kwargs)
    except FileNotFoundError:
        logger.error(f"File not found: {excel_abfss_path}")
        raise
    except ValueError as e:
        logger.error(f"ValueError when reading Excel: {e}")
        raise

    # Only convert to Arrow after reading succeeded, outside try block
    return pa.Table.from_pandas(pandas_df, preserve_index=preserve_index)


def safe_api_call(func, retries=3, delay=2, **kwargs):
    """Safely calls a function with retry logic."""
    for attempt in range(retries):
        try:
            return func(**kwargs)
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                raise